# Global indicator project
## Create a sausage buffer for each sample point to identify walkable local neighborhood

This notebook creates sausage buffer to determine local walkable neighborhood (1600m sample points + 50m walkable street network buffer).

** Buffering technique referring to Forsyth et al [here](https://ij-healthgeographics.biomedcentral.com/articles/10.1186/1476-072X-11-14). Mavoa et al 2018, see [here](https://www.jtlu.org/index.php/jtlu/article/view/1132)   
** load the projected graph, using the projected graph to get network stats so that the ouput unit is m/km

In [1]:
# Libraries used for OSMnx analyses and output
import networkx as nx
import time 
import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np
import requests
import pandas as pd
import geopandas as gpd
import fiona

from descartes import PolygonPatch
from shapely.geometry import shape,Point, LineString, Polygon


ox.config(use_cache=True, log_console=True)

### Set up project config

In [2]:
place = 'phoenix' 

region = 'Arizona, USA' # study region name

studyregion = 'Phoenix, Arizona, USA'

suffix = '_201905' # output data time

buffer_dist = 1e4 #study region buffer 10km

point_dist = 30 #sample point interval distance

network_type='walk'

OSM_folder = '../data/OSM'

G_filename = 'Phoenix, Arizona, USA_walk_201905.graphml'

point_filepath = '../data/OSM/phoenix_testing_sample_points_201905/phoenix_testing_sample_points_201905.shp'

## Load pedestrain street network and sample points from local folder

In [15]:
G = ox.load_graphml(filename=G_filename, folder=OSM_folder)

In [7]:
G_proj = ox.project_graph(G)
ox.save_graphml(G_proj, filename='{studyregion}_proj_{network_type}{suffix}.graphml'.format(
        studyregion = place, network_type=network_type, suffix = suffix), folder=OSM_folder)

In [3]:
# load the projected network graph
G_proj = ox.load_graphml(filename='phoenix_proj_walk_201905.graphml', folder=OSM_folder)

In [5]:
# load sample point shapefile
sample_points = gpd.GeoDataFrame.from_file(point_filepath)

# create list of sample points to iterate over
point_locations = []

for point in sample_points.geometry: 
    point = (point.x, point.y)
    point_locations = point_locations + [point]

## Create sausage buffer

In [6]:
# create sausage buffer local neighborhood graph
def create_sausage_buffer_G(G_proj, orig_point, buffer=50, length = 1600, intersection_tolerance = 15):
    # locate closest node on network to 
    orig_node = ox.get_nearest_node(G_proj, orig_point, return_dist=True)
    subgraph_proj = nx.ego_graph(G_proj, orig_node[0], radius=length, distance='length')
    # create buffer
    subgraph_gdf = ox.graph_to_gdfs(subgraph_proj, nodes=False, edges=True, fill_edge_geometry=True)
    buffer = subgraph_gdf.geometry.buffer(buffer)
    buffer_uu = buffer.geometry.unary_union
    return([subgraph_proj, buffer_uu]) #output is a list of sample point graphs (1600m), and buffer polygons

In [ ]:
# create sausage buffer local neighborhood geodataframe
def create_sausage_buffer_gdf(G_proj, orig_point, buffer=50, length = 1600, intersection_tolerance = 15):
    # locate closest node on network to 
    orig_node = ox.get_nearest_node(G_proj, orig_point, return_dist=True)
    subgraph_proj = nx.ego_graph(G_proj, orig_node[0], radius=length, distance='length')
    subgraph_gdf = ox.graph_to_gdfs(subgraph_proj, nodes=False, edges=True, fill_edge_geometry=True)
    # create buffer polygon geometry to dataframe
    subgraph_gdf['geometry'] = subgraph_gdf.geometry.buffer(buffer) 
    #link original node id reference
    subgraph_gdf['node_id'] = orig_node[0]
    return(subgraph_gdf) #output is smaple point subgraph with buffer polygon geometry and original node id reference

In [7]:
start = time.time()
task = 'Buffer network for {} sample points'.format(len(point_locations))
sausagebuffers = []
for point in point_locations:
    sausagebuffers.append(create_sausage_buffer_G(G_proj, point))
print('Completed task "{}" in {:,.2f} seconds'.format(task,time.time() - start)) 

Completed task "Buffer network for 100 sample points" in 245.65 seconds


## Get sample point neighborhood stats using OSMnx

In [10]:
# Local neighbourhood analysis
def analyse_local_nh(G_proj, orig_point):
    buffer = create_sausage_buffer_G(G_proj, orig_point)
    orig_node = ox.get_nearest_node(G_proj, orig_point, return_dist=True)
    #get stats
    area_sqm = buffer[1].area
    area_sqkm = area_sqm*1e-06
    stats = ox.basic_stats(buffer[0], area=area_sqm, clean_intersects=True, circuity_dist='euclidean')
    return({ 'origin_node_id': orig_node[0],
             'area_sqkm': area_sqkm,
             'stats': stats,
             'origin_node_snap_dist': orig_node[1]})
     

In [11]:
# initialise a list
start = time.time()
task = "Calculate local neighbourhood measures for {} points".format(len(point_locations))
nh_estimates = []
for point in point_locations:
    nh_estimates.append(analyse_local_nh(G_proj, point))
    
# area in sqkm
area_km = [x['area_sqkm'] for x in nh_estimates]
# clean intersection density per sqkm
clean_intersection_count = [x['stats']['clean_intersection_count'] for x in nh_estimates]
# clean intersection density
clean_intersection_density_km = [x['stats']['clean_intersection_density_km'] for x in nh_estimates]
# dist to the nodes
origin_node_snap_dist = [x['origin_node_snap_dist'] for x in nh_estimates]
# nearest node id
origin_node_id = [x['origin_node_id'] for x in nh_estimates]

#load sample point dataframe and record new stats
sample_points['area_km'] = area_km
sample_points['intct_count'] = clean_intersection_count
sample_points['intct_den']= clean_intersection_density_km
sample_points['node_dist']= origin_node_snap_dist
sample_points['node_id']= origin_node_id


print('Completed task "{}" in {:,.2f} seconds'.format(task,time.time() - start)) 
  

Completed task "Calculate local neighbourhood measures for 100 points" in 403.41 seconds


In [14]:
sample_points.columns

Index(['index', 'Series', 'access', 'area', 'bridge', 'highway', 'junction',
       'key', 'lanes', 'length', 'maxspeed', 'name', 'oneway', 'osmid', 'ref',
       'service', 'tunnel', 'u', 'v', 'width', 'points', 'geometry', 'area_km',
       'intct_count', 'intct_den', 'node_dist', 'node_id'],
      dtype='object')

In [15]:
# save sample point stats dataframe
sample_points.to_csv('../data/OSM/phoenix_sample_points_stats_201905.csv')